# Deploying Python bot on Facebook Messenger

Let's say you have a chatbot written in Python and you want it available for many people to use. Why not deploy it on Facebook Messenger? 
I will show you below how you can make necessary set ups in this tutorial.

## Step 1. Installing neccessary modules and program

You need three main things to connect your Python code to Messenger:
1. flask [link](http://flask.pocoo.org/)
2. requests [link](https://2.python-requests.org//en/master/)
3. ngrok [link](https://ngrok.com/product)

You can install them by following the link above. If you don't have them already, you are mostly likely missing other modules required to run them. You need to install all the modules to run the Python code I will post below. 

Before that, let me explain the purpose of three main modules/program. `flask` is a module used to create local server in your computer that can receive messages from Messenger. `ngrok` is a program that will forward a http connection from Messenger to your computer. Lastly, `requests` is a function that will send the reply from your bot back to Messenger. 

Now, save the following Python code in `server.py` and put it inside a directory where your chatbot code is located. 

In [ ]:
from flask import Flask, request
import requests

app = Flask(__name__)

FB_API_URL = 'https://graph.facebook.com/v2.6/me/messages'
VERIFY_TOKEN = ''# <paste your verify token here>
PAGE_ACCESS_TOKEN = ''# paste your page access token here>"

@app.route("/webhook",methods=['GET','POST'] )
def listen():
    """This is the main function flask uses to 
    listen at the `/webhook` endpoint"""
    if request.method == 'GET':
        return verify_webhook(request)

    if request.method == 'POST':
        payload = request.json
        event = payload['entry'][0]['messaging']
        for x in event:
            if is_user_message(x):
                text = x['message']['text']
                sender_id = x['sender']['id']
                respond(sender_id, text)

        return "ok"

def verify_webhook(req):
    if req.args.get("hub.verify_token") == VERIFY_TOKEN:
        return req.args.get("hub.challenge")
    else:
        return "incorrect"

def respond(sender, message):
    """Formulate a response to the user and
    pass it on to a function that sends it."""
    response = get_bot_response(message)
    send_message(sender, response)

    
def get_bot_response(message):
    """This is just a dummy function, returning a variation of what
    the user said. Replace this function with one connected to chatbot."""
    return "This is a dummy response to '{}'".format(message)


def is_user_message(message):
    """Check if the message is a message from the user"""
    return (message.get('message') and
            message['message'].get('text') and
            not message['message'].get("is_echo"))

def send_message(recipient_id, text):
    """Send a response to Facebook"""
    payload = {
        'message': {
            'text': text
        },
        'recipient': {
            'id': recipient_id
        },
        'notification_type': 'regular'
    }

    auth = {
        'access_token': PAGE_ACCESS_TOKEN
    }

    response = requests.post(
        FB_API_URL,
        params=auth,
        json=payload
    )

    return response.json()

## Step 2. Understanding code to access Messenger messages and send them to your computer

Now I will explain what above code does in detail. `flask` will create `app` that will listen for http requests of messages sent to `localhost:5000/webhook`, which is your localhost server. 

Inside `listen()` function, `request` function will use `verify_webhook(req)` function to authenticate connection between your `app` and Facebook. 

Once `listen()` function sees that message from Messenger is valid, it will send it to `respond(sender, message)` function, which access bot's code directly through `get_bot_response(message)` function. 

Finally, `send_message(recipient_id, text)` will send bot's response back to Messenger. 

Note that `VERIFY_TOKEN` and `PAGE_ACCESS_TOKEN` have been left out. `VERIFY_TOKEN` is any string you would be want to use as password to let Facebook know your localhost server wants to listen for messages. We will be coming back for `PAGE_ACCESS_TOKEN` later. 

## Step 3. Running ngrok to start localhost server

Once you installed ngrok program in your computer, start the program and don't turn it off while setting up connection. In terminal window, type `ngrok http 5000`. This will set up a http endpoint that will be forwarded to your computer on port 5000. Your http endpoint will look something like, `http://9cbec3d0.ngrok.io`. 

![Imgur](https://i.imgur.com/Fs4QDvi.png)

## Step 4. Setting up Facebook page and getting Page Access Token

Let's head over to Facebook Developer website by clicking this [link](https://developers.facebook.com/). If you already have a Facebook account, just simply log in.  

Now, click on `My Apps` on the top right corner and click `Create an App`.

![Imgur](https://i.imgur.com/GPlijTg.png)

Create the display name of your App and click `Create App ID`. You have to do a quick security check to prove that you are not a robot. The display name can be anything you choose to be. I choose to use Bookbot for the demonstration purpose. 

![Imgur](https://i.imgur.com/1Sc0QNx.png)

You will be lead to your bot's dashboard. 

![Imgur](https://i.imgur.com/X5ixdCv.png)

First, click `Skip` for the first page you see like in the image above.

Then, if you scroll down, you will see `Add a Product` section. Click on `Set Up` button for Messenger. 

In the next page you see, scroll down until you see `Access Tokens` section. You don't have Page Access Token yet for your chatbot because you don't have a Facebook page yet. What you want to do is create a new page. 

![Imgur](https://i.imgur.com/6QJecWp.png)

When you click on `Create a New Page`, you will be led to the following page:

![Imgur](https://i.imgur.com/tSz1Hid.png)

Click on either options to create a page. I have decided to create my BookBot page as a community page but it is up to you. 

![Imgur](https://i.imgur.com/FAXRtUx.png)

Once you click through a few options for profile picture, you will finally be able to see your bot's Facebook page. 

![Imgur](https://i.imgur.com/E9sVpEs.png)

Now, time to get back to Facebook Developer page for your bot. When you refresh the page, you can now select your bot's page to get Access Token. But you are not quite there yet. 

![Imgur](https://i.imgur.com/S9pb3Kp.png)

You will be told that you have to edit your permission to get access token. Just click on `Edit Permissions` and follow along the instruction until your bot is linked to the Facebook page. 

Now, you will see Page Access Token appear. 

![Imgur](https://i.imgur.com/LqoOa5W.png)

Click on it to copy to a clipboard and head over to `server.py`. 

## Step 5. Starting chatbot server

First, copy and paste Page Access Token to `PAGE_ACCESS_TOKEN` in `server.py`. Save the file.

Second, in a separate terminal window, `cd` to the directory where you have your chatbot code and `server.py` code. 

Third, in the terminal window, enter `set FLASK_APP=server.py`. Then enter `flask run`. 

If everything goes well, you will see something similar to following screen:

![Imgur](https://i.imgur.com/LSp8HHT.png)

## Step 6. Setting up webhook

Go to ngrok server and copy the address starting with `https`. Now go over to Facebook Development page for your bot. 
Underneath `Access Tokens` section, you will see `Webhooks` section. Click `Subscribe to Event`. Then copy and paste the address you copied in `Callback URL` section. Make sure to end the address with `/webhook`, because this is how your flask app will find webhook address. Ignore the callback URL in the following image. 

![Imgur](https://i.imgur.com/Qu8BMF5.png)

Next, fill out `Verify Token` with `VERIFY_TOKEN` from `server.py`. 

Finally, click `messages` and `messages_postbacks` from Subscription Fields and click `Verify and Save` button. 

Lastly, you want to choose the page you want to subscribe to and click `Subscribe`. 

![Imgur](https://i.imgur.com/hCArdlN.jpg)

Now you are all set! How do you know?

![Imgur](https://i.imgur.com/EHxCGbS.jpg)

On the navigation pane to the left, if you see the word `Webhook` with a green circle and a check mark, that means all the set up has been successfully completed. 

## Step 7. Test your chatbot!

Now, if you didn't get any error, everything has been set up! 

Once you are over to your bot's Facebook page, click on three dot icons and click `View as Page Visitor`.

![Imgur](https://i.imgur.com/oFDOtGV.jpg)

Now the moment of the truth! Click on `Send Message` and when the chatbox pops up start tying in question you have trained your bot for!

![Imgur](https://i.imgur.com/LEgUYae.jpg)

Open the chatbox in Messenger mode and here is what you will see. *Note: BookBot you see in the examples above has been changed to Testbot because of working issue.* 

![Imgur](https://i.imgur.com/tDlfcRz.png)

You can see the dummy chatbot code from `server.py` showing up on Messenger chatbox. If you have gotten this far, congratulations!

Last but not least, you probably want your chatbot to do more than simply repeat what the user says. You can use the following simple chatbot code to simulate conversation. Copy and save the following as `chatbot.py` and make sure it is in the same directory as `server.py`. 

In [ ]:
import re
import random

rules = {"I want (.*)":["What would it mean if you got {0}", 
                        "Why do you want {0}", 
                        "What's stopping you from getting {0}"], 
         "if (.*)":["Do you really think it's likely that {0}", 
                    "Do you wish that {0}", 
                    "What do you think about {0}", 
                    "Really--if {0}"], 
         "do you think (.*)":["{0} Absolutely.", 
                              "No chance"], 
         "do you remember (.*)":["Did you think I would forget {0}", 
                                 "Why haven't you been able to forget {0}", 
                                 "What about {0}", 
                                 "Yes .. and?"]
        }

# Define respond()
def robo_respond(message):
    # Call match_rule
    response, phrase = match_rule(rules, message)
    if '{0}' in response:
        # Replace the pronouns in the phrase
        phrase = replace_pronouns(phrase)
        # Include the phrase in the response
        response = response.format(phrase)
    return response

# Define match_rule()
def match_rule(rules, message):
    response, phrase = "default", None
    
    # Iterate over the rules dictionary
    for pattern, responses in rules.items():
        # Create a match object
        match = re.search(pattern, message)
        if match is not None:
            # Choose a random response
            response = random.choice(responses)
            if '{0}' in response:
                phrase = match.group(1)
    # Return the response and phrase
    return response, phrase

def replace_pronouns(message):
    message = message.lower()
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub('me', 'you', message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub('my', 'your', message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub('your', 'my', message)
    if 'you' in message:
        # Replace 'you' with 'I'
        return re.sub('you', 'I', message)
    return message


To make this new code work, you have to go back to `server.py` and add `from chatbot import robo_respond` at the top. Also change return value of `get_bot_response(message)` to `return robo_respond(message)`. 

Finally, you have to restart chatbot server (Step 5) and refresh your chatbot's Facebook page. The response from the bot you see should have changed based on new code you imported.

![Imgur](https://i.imgur.com/YWK8ZB3.png)

## Step. 8 Last Word

Now you have learned how to set up connection between your chatbot Python code and Facebook Messenger. You can now focus on improving the given chatbot code to answer more intelligently. Good luck!